In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

df = pd.read_excel('cancer patient data sets.xlsx')

In [3]:
for column in df.columns:
  if column not in ['Patient Id','Age','Gender','Level']:
    max_val = df[column].max()
    df[column] = (df[column] / max_val * 5).astype(int)
df

,Patient Id,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,Occupational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swallowing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,P1,33,1,1,2,3,2,2,1,1,...,1,2,1,1,1,0,1,2,2,Low
1,P10,17,1,1,0,3,1,2,1,1,...,0,1,3,5,3,1,0,5,1,Medium
2,P100,35,1,2,3,3,3,3,2,4,...,4,4,5,1,0,2,4,5,1,High
3,P1000,37,1,4,4,4,4,4,5,5,...,2,1,1,0,2,2,4,5,3,High
4,P101,46,1,3,5,4,4,5,4,5,...,1,1,2,0,2,1,2,1,2,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,P995,44,1,3,4,4,4,5,4,5,...,2,1,1,4,5,1,2,3,2,High
996,P996,37,2,3,5,4,4,5,4,5,...,5,3,2,4,1,2,2,0,2,High
997,P997,25,2,2,3,3,3,3,2,4,...,4,4,5,1,0,2,4,5,1,High
998,P998,18,2,3,5,4,4,5,4,5,...,1,1,2,0,2,1,2,1,2,High


In [4]:
def scale_df(df):
  X = df[df.columns[:-1]].values
  y = df[df.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  data = np.hstack((X,np.reshape(y,(len(y),1))))
  return data,scaler

In [5]:
def convert(str):
  return 1 if str == "Low" else 2 if str == "Medium" else 3
df.drop(['Patient Id','Air Pollution','chronic Lung Disease','Swallowing Difficulty','Clubbing of Finger Nails'], axis=1, inplace=True)
df['Level'] = df['Level'].apply(convert)
cols = df.columns
df,scaler = scale_df(df)
df = pd.DataFrame(df,columns=cols)
df

,Age,Gender,Alcohol use,Dust Allergy,Occupational Hazards,Genetic Risk,Balanced Diet,Obesity,Smoking,Passive Smoker,Chest Pain,Coughing of Blood,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Frequent Cold,Dry Cough,Snoring,Level
0,-0.347848,-0.819903,-0.269967,0.173908,-0.420403,-0.596676,-1.144835,-0.500259,-0.592832,-0.783927,-0.847870,-0.164916,-0.580568,0.022926,-0.751623,-0.711626,-0.764392,-0.200129,0.357191,1.0
1,-1.681238,-0.819903,-1.390160,0.173908,-1.124595,-0.596676,-1.144835,-1.076594,-0.592832,-0.138188,-0.847870,-0.914531,-1.363004,-0.671801,0.819167,2.287850,-1.461194,1.652915,-0.551693,2.0
2,-0.181174,-0.819903,0.290130,0.173908,0.283789,-0.002969,0.630103,1.228747,-0.592832,-0.783927,0.036249,1.334316,1.766741,1.412381,2.389958,-0.711626,1.326015,1.652915,-0.551693,3.0
3,-0.014501,-0.819903,0.850227,0.930032,0.987981,0.590739,1.221749,1.228747,1.194601,1.153290,0.920368,1.334316,0.201869,-0.671801,-0.751623,-1.461495,1.326015,1.652915,1.266075,3.0
4,0.735531,-0.819903,1.410323,0.930032,0.987981,1.184446,1.221749,1.228747,1.790412,1.153290,0.920368,2.083932,-0.580568,-0.671801,0.033772,-1.461495,-0.067590,-0.817810,0.357191,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.568858,-0.819903,0.850227,0.930032,0.987981,1.184446,1.221749,1.228747,1.194601,1.799029,0.920368,0.584700,0.201869,-0.671801,-0.751623,1.537981,-0.067590,0.417553,0.357191,3.0
996,-0.014501,1.219657,1.410323,0.930032,0.987981,1.184446,1.221749,1.228747,1.194601,1.799029,0.920368,0.584700,2.549177,0.717653,0.033772,1.537981,-0.067590,-1.435491,0.357191,3.0
997,-1.014543,1.219657,0.290130,0.173908,0.283789,-0.002969,0.630103,1.228747,-0.592832,-0.783927,0.036249,1.334316,1.766741,1.412381,2.389958,-0.711626,1.326015,1.652915,-0.551693,3.0
998,-1.597901,1.219657,1.410323,0.930032,0.987981,1.184446,1.221749,1.228747,1.790412,1.153290,0.920368,2.083932,-0.580568,-0.671801,0.033772,-1.461495,-0.067590,-0.817810,0.357191,3.0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Level', axis=1), df['Level'], shuffle=True,
    test_size=0.2, random_state=42)

svm_model = SVC()
svm_model = svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [7]:
import joblib

joblib.dump(svm_model, 'svm_model.pkl')

['svm_model.pkl']

In [11]:
model = joblib.load('svm_model.pkl')
arr = scaler.transform([[23,1,3,1,3,5,2,3,5,2,4,5,2,1,2,2,4,3,2]])
y_pred=model.predict(arr)
int(y_pred[0])

print("High" if int(y_pred[0]) == 3 else "Medium" if int(y_pred[0]) == 2 else "Low")

Medium


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
